<a href="https://colab.research.google.com/github/Harrow-Enigma/spring-2022/blob/main/gradient-descent-demo/Line_of_best_fit_with_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Enigma Logo" height="100px" src="https://avatars.githubusercontent.com/u/74505663?v=4" align="left" hspace="10px" vspace="0px"></p>

# Gradient Descent Demo: Line of Best Fit
*By Team Enigma*

In [ ]:
#@title Copyright 2022 Team Enigma, licensed under the GNU GPL v3 License

print("""
Code illustrating gradient descent by iteratively updating a line of best fit.
Copyright (C) 2022  Team Enigma

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
""")

## Setting Up

In [ ]:
#@title Importing Libraries

import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#@title Plotting Utilities

x_vals = np.arange(0, 10, 0.1)

def return_linear_plot_obj(m, c):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x_vals, m * x_vals + c)

    ax.set_xlim([0, 10])
    ax.set_ylim([0, 20])

    return fig

def plot_linear(m, c):
    return_linear_plot_obj(m, c)
    plt.show()

def get_linear_img(m, c):
    fig = return_linear_plot_obj(m, c)
    fig.canvas.draw()

    img = np.fromstring(
        fig.canvas.tostring_rgb(), dtype=np.uint8, sep=''
    )
    img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    height, width, _ = img.shape

    fig.clear()

    return img, width, height

def create_video(fname, frames, w, h):
    video = cv2.VideoWriter(fname, cv2.VideoWriter_fourcc(*'MP4V'), 30, (w, h))
    for f in frames:
        video.write(f)
    cv2.destroyAllWindows()
    video.release()

## Target Function and Data Generation

In [ ]:
M, C = 1.5, 2

print(f'Target function: m={M}, c={C}\n')

def target_function(inp):
    return M * inp + C

y_vals = target_function(x_vals)

plot_linear(M, C)

## Single Perceptron and Gradients

In [ ]:
def perceptron(x, m, c):
    return m * x + c

def perceptron_grad(x, m, c):
    return {'m': x, 'c': 1}

## Loss Function and Gradients

In [ ]:
def loss(y_target, y):
    if y > y_target:
        return y - y_target
    if y < y_target:
        return y_target - y
    if y == y_target:
        return 0

def loss_grad(y_target, y):
    if y > y_target:
        return 1
    if y < y_target:
        return -1
    if y == y_target:
        print('Error - gradient = 0')
        return 0

In [ ]:
def apply_gradients(val, grad, lr):
    return val - lr * grad

## Approximation Initialisation and Visualisation

In [ ]:
m, c = 1, 1

print(f'Approximate function initialisation: m={m}, c={c}\n')

plot_linear(m, c)

In [ ]:
x_test = 2
targ = target_function(x_test)
out = perceptron(x_test, m, c)
print(f'At x={x_test}, output={out}, ground truth={targ}, loss={loss(targ, out)}\n')

scout = 0.5
m_neighbours = np.arange(m-scout, m+scout, 0.01)
m_neighbour_losses = [loss(targ, perceptron(x_test, n, c)) for n in m_neighbours]

plt.plot(m_neighbours, m_neighbour_losses)
plt.xlabel('m')
plt.ylabel('loss')
plt.title(f'How the loss changes as m changes in the neighbourhood of x={x_test}')
plt.show()

## Training

In [ ]:
learning_rate = 0.001
epochs = 100

frames, w, h = [], 0, 0

for ep in range(100):
    total_loss = []

    for e, (i, t) in enumerate(zip(x_vals, y_vals)):
        # computing outputs
        y = perceptron(i, m, c)
        l = loss(t, y)

        # gradient propagating
        l_grad = loss_grad(t, y)
        y_grad = perceptron_grad(i, m, c)
        m_grad = y_grad['m'] * l_grad     # chain rule
        c_grad = y_grad['c'] * l_grad     # chain rule

        # gradient update
        m = apply_gradients(m, m_grad, learning_rate)
        c = apply_gradients(c, c_grad, learning_rate)

        total_loss.append(l)
        if e % ((10*10)/(epochs*len(x_vals))) == 0:
            img, w, h = get_linear_img(m, c)
            frames.append(img)
    
    print(f'Epoch {ep+1} mean loss: {np.mean(total_loss)}; m={m}, c={c}')

### Result Visualisation

In [ ]:
print(f'Approximate function result: m={m}, c={c}\n')

plot_linear(m, c)

In [ ]:
create_video('convergence.mp4', frames, w, h)